In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
-- 1. Create the database and schema for GraphGuard
CREATE DATABASE IF NOT EXISTS GRAPHGUARD_DB;
USE DATABASE GRAPHGUARD_DB;

CREATE SCHEMA IF NOT EXISTS TRANSACTIONS;
USE SCHEMA TRANSACTIONS;

In [ ]:
-- 2. Create the Raw Transactions Table
CREATE TABLE IF NOT EXISTS RAW_TRANSACTIONS (
    TRANSACTION_ID VARCHAR,
    USER_ID VARCHAR,
    CARD_ID VARCHAR,
    TXN_YEAR INT,
    TXN_MONTH INT,
    TXN_DAY INT,
    TXN_TIME VARCHAR,
    AMOUNT FLOAT,
    MERCHANT_NAME VARCHAR,
    MERCHANT_CITY VARCHAR,
    MERCHANT_STATE VARCHAR,
    MERCHANT_ZIP VARCHAR,
    MCC_CODE VARCHAR,
    IS_FRAUD BOOLEAN
);

In [ ]:
-- 3. Create a view that automatically structures recent valid transactions for our LLM
CREATE OR REPLACE VIEW RECENT_USER_ACTIVITY AS
SELECT 
    USER_ID,
    MERCHANT_NAME,
    MERCHANT_CITY,
    AMOUNT,
    TXN_YEAR || '-' || TXN_MONTH || '-' || TXN_DAY AS TXN_DATE
FROM RAW_TRANSACTIONS
WHERE IS_FRAUD = FALSE
ORDER BY TXN_YEAR DESC, TXN_MONTH DESC, TXN_DAY DESC;

In [ ]:
-- 4. Insert dummy data to simulate the Kaggle dataset
INSERT INTO RAW_TRANSACTIONS (TRANSACTION_ID, USER_ID, TXN_YEAR, TXN_MONTH, TXN_DAY, AMOUNT, MERCHANT_NAME, MERCHANT_CITY, IS_FRAUD)
VALUES 
('TXN1', 'USER_9981', 2026, 2, 21, 54.30, 'Kroger', 'Columbus, OH', FALSE),
('TXN2', 'USER_9981', 2026, 2, 20, 12.50, 'Uber', 'Columbus, OH', FALSE),
('TXN3', 'USER_4412', 2026, 2, 22, 8.99, 'Starbucks', 'Seattle, WA', FALSE);

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import json

# Snowflake Notebooks automatically provide a 'session' object
session = snowpark.Session.builder.getOrCreate()

def get_user_transaction_graph(user_id: str) -> str:
    """Fetches recent transactions from the Snowflake view to build GraphRAG context."""
    
    # Query the Feature View using Snowpark
    df = session.table("GRAPHGUARD_DB.TRANSACTIONS.RECENT_USER_ACTIVITY") \
                .filter(col("USER_ID") == user_id) \
                .limit(5)
    
    # Convert to Pandas for easy JSON serialization
    pdf = df.to_pandas()
    
    if pdf.empty:
        return json.dumps({"user_id": user_id, "error": "No recent valid transactions found."})
    
    # Build the 'Nodes' for the GraphRAG
    recent_nodes = []
    for _, row in pdf.iterrows():
        recent_nodes.append({
            "merchant": row["MERCHANT_NAME"],
            "location": row["MERCHANT_CITY"],
            "amount": float(row["AMOUNT"]),
            "date": row["TXN_DATE"]
        })
        
    graph_context = {
        "user_id": user_id,
        "recent_nodes": recent_nodes
    }
    
    return json.dumps(graph_context, indent=2)

# --- Test the Pipeline ---
print("--- GRAPHGUARD FEATURE EXTRACTION TEST ---")
test_context = get_user_transaction_graph("USER_9981")
print(f"\nExtracted JSON Context for LLM prompt:\n{test_context}")